In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
import glob
import torch
import utils
from scipy.special import erf
from sklearn.calibration import calibration_curve
from sklearn.metrics import (auc, brier_score_loss, confusion_matrix,
                             mean_absolute_error, mean_squared_error,
                             precision_recall_curve, r2_score, roc_curve,
							 precision_score, recall_score, f1_score)

In [ ]:
model_type = 'swmag'
shap_dir = 'outputs/shap_values'
scaler_dir = 'outputs/scalers'
results_dir = 'outputs/rsd'
VERSION = 'swmag_v6-1'
shap_files = glob.glob(f'{shap_dir}/*{VERSION}*.pkl')
scaler_files = glob.glob(f'{scaler_dir}/*{VERSION}.pkl')
results_files = glob.glob(f'{scaler_dir}/*{VERSION}.feather')
with open('cluster_dict.pkl', 'rb') as f:
	cluster_dict = pickle.load(f)

In [ ]:
cluster_dict['greenland_cluster']['regions']['GRL-0'].keys()

In [ ]:
results = {}
for cluster in cluster_dict.values():
	for key, region in cluster['regions'].items():
		results[key] = {}
		results[key]['swmag_results'] = pd.read_feather(f'{results_dir}/non_twins_modeling_region_{key}_version_swmag_v6-1.feather')
		results[key]['maxpool_results'] = pd.read_feather(f'{results_dir}/twins_modeling_region_{key}_version_twins_v_maxpooling.feather')
		results[key]['mean_lat'] = utils.getting_mean_lat(region['stations'])

In [ ]:
swmag_auc, maxpool_auc, mean_lat = [], [], []
for key in results.keys():

	prec, rec, __ = precision_recall_curve(y_true=results[key]['swmag_results']['actual'], probas_pred=results[key]['swmag_results']['predicted_mean'])
	swmag_auc.append(auc(rec, prec))
	prec, rec, __ = precision_recall_curve(y_true=results[key]['maxpool_results']['actual'], probas_pred=results[key]['maxpool_results']['predicted_mean'])
	maxpool_auc.append(auc(rec, prec))
	mean_lat.append(results[key]['mean_lat'])


In [ ]:
swmag_plot_df = pd.DataFrame({'mean_lat':mean_lat, 'swmag_auc':swmag_auc})
maxpool_plot_df = pd.DataFrame({'mean_lat':mean_lat, 'maxpool_auc':maxpool_auc})

swmag_plot_df.sort_values(by='mean_lat', inplace=True)
maxpool_plot_df.sort_values(by='mean_lat', inplace=True)

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(10,7))
axes.scatter(swmag_plot_df['mean_lat'], swmag_plot_df['swmag_auc'], label='swmag', color='blue')
axes.scatter(maxpool_plot_df['mean_lat'], maxpool_plot_df['maxpool_auc'], label='maxpool', color='orange')

swmag_z = np.polyfit(swmag_plot_df['mean_lat'], swmag_plot_df['swmag_auc'], 4)
swmag_p = np.poly1d(swmag_z)
maxpool_z = np.polyfit(maxpool_plot_df['mean_lat'], maxpool_plot_df['maxpool_auc'], 4)
maxpool_p = np.poly1d(maxpool_z)

axes.plot(swmag_plot_df['mean_lat'], swmag_p(swmag_plot_df['mean_lat']), color='blue')
axes.plot(maxpool_plot_df['mean_lat'], maxpool_p(maxpool_plot_df['mean_lat']), color='orange')

axes.set_xlabel('Mean Region MLAT')
axes.set_ylabel('PR-AUC Score')
axes.set_title('PR-AUC vs Region MLAT')
axes.legend()
plt.tight_layout()
plt.savefig(f'plots/pr-auc_vs_region_mlat_swmag_maxpool.png')

In [ ]:
def plotting_continuious_reliability_diagram(results, model):

	''' Function that plots the reliability diagram for the predictions.'''

	x = np.linspace(0, 1, 1000)

	fig, ax = plt.subplots(ncols=1, nrows=2, sharex=True, figsize=(10,15))

	for region in results.keys():
		predictions = results[region][f'{model}_results'].dropna(inplace=False, subset=['actual', 'predicted_mean', 'predicted_std'])
		actual = predictions['actual']
		predicted_mean = predictions['predicted_mean']
		predicted_std = predictions['predicted_std'].abs()

		standard_error = (actual - predicted_mean)/(np.sqrt(2) * predicted_std).to_numpy() #Standard error for each parameter
		cumulative_dist = np.zeros((len(x), 1)) #Cumulative distribution for each parameter
		for i in standard_error.index:
			cumulative_dist[:,0] += (1/len(standard_error)) * np.heaviside(x - 0.5*(erf(standard_error.loc[i])+1) , 1) #Calculate the cumulative distribution for each parameter

		ax[0].plot(x, cumulative_dist[:,0], label=region)
		ax[1].plot(x, x - cumulative_dist[:,0], label=region)

	#Place legend to the right middle of the figure
	ax[0].legend(bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
	ax[0].plot(x, x, linestyle = '--', color = 'k')
	ax[0].set_ylabel('Observed Frequency')
	ax[0].set_xlim(0,1)
	ax[0].set_ylim(0,1)
	fig.suptitle(f'{model} Reliability Diagram')

	ax[1].plot(x, np.zeros(len(x)), linestyle = '--', color = 'k')
	ax[1].set_ylim(-0.2,0.2)
	ax[1].set_xlabel('Predicted Frequency')
	ax[1].set_ylabel('Under/Over-\nEstimation')
	ax[1].set_aspect('equal')
	plt.subplots_adjust(hspace = -0.20)

	plt.tight_layout()
	plt.savefig(f'plots/{model}_reliability_diagram.png', bbox_inches='tight')

In [ ]:
plotting_continuious_reliability_diagram(results, model='swmag')
plotting_continuious_reliability_diagram(results, model='maxpool')

In [ ]:
scalers = {}
for cluster in cluster_dict.values():
	for key, region in cluster['regions'].items():
		if os.path.exists(f'{scaler_dir}/{model_type}_{key}_{VERSION}.pkl'):
			with open(f'{scaler_dir}/{model_type}_{key}_{VERSION}.pkl', 'rb') as f:
				scaler_values = pickle.load(f)
				scalers[key] = scaler_values

In [ ]:
for cluster in cluster_dict.values():
	for key, region in cluster['regions'].items():
		if os.path.exists(f'{shap_dir}/{model_type}_region_{key}_{VERSION}.pkl'):
			with open(f'{shap_dir}/{model_type}_region_{key}_{VERSION}.pkl', 'rb') as f:
				shap_values = pickle.load(f)
				cluster['regions'][key]['shap_dict'] = shap_values

In [ ]:
# individual_cluster_to_examine = 'canadian_cluster'
mean_shap, std_shap, X, mu, sigma, error = [], [], [], [], [], []
for cluster in cluster_dict.keys():
	for reg, region in cluster_dict[cluster]['regions'].items():
		if 'shap_dict' in region.keys():

			region['shap_dict']['mean_shap'] = np.concatenate([region['shap_dict']['shap_values'][i][0][:,:,:,:] for i in range(len(region['shap_dict']['shap_values']))], axis=0)
			region['shap_dict']['std_shap'] = np.concatenate([region['shap_dict']['shap_values'][i][1][:,:,:,:] for i in range(len(region['shap_dict']['shap_values']))], axis=0)

			for key in region['shap_dict'].keys():
				if isinstance(region['shap_dict'][key], torch.Tensor):
					region['shap_dict'][key] = region['shap_dict'][key].cpu().detach().numpy()

			region['shap_dict']['mean_shap'] = region['shap_dict']['mean_shap'].reshape(region['shap_dict']['mean_shap'].shape[0], region['shap_dict']['mean_shap'].shape[2], region['shap_dict']['mean_shap'].shape[3])
			region['shap_dict']['std_shap'] = region['shap_dict']['std_shap'].reshape(region['shap_dict']['std_shap'].shape[0], region['shap_dict']['std_shap'].shape[2], region['shap_dict']['std_shap'].shape[3])
			region['shap_dict']['xtest'] = region['shap_dict']['xtest'].reshape(region['shap_dict']['xtest'].shape[0], region['shap_dict']['xtest'].shape[2], region['shap_dict']['xtest'].shape[3])

			mean_added = np.sum(np.sum(np.abs(region['shap_dict']['mean_shap']), axis=1),axis=1)
			region['shap_dict']['mean_shap'] = region['shap_dict']['mean_shap']/mean_added[:,None,None]

			std_added = np.sum(np.sum(np.abs(region['shap_dict']['std_shap']), axis=1),axis=1)
			region['shap_dict']['std_shap'] = region['shap_dict']['std_shap']/std_added[:,None,None]

			region_scaler = scalers[reg]['swmag_scaler']

			unbroadcasted_mu = results[reg]['swmag_results']['predicted_mean'].to_numpy()
			unbroadcasted_sigma = results[reg]['swmag_results']['predicted_std'].to_numpy()
			unbroadcasted_error = np.array(np.abs(results[reg]['swmag_results']['predicted_mean'] - results[reg]['swmag_results']['actual']))

			concatenated_mu = np.concatenate([np.concatenate([unbroadcasted_mu[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)
			concatenated_sigma = np.concatenate([np.concatenate([unbroadcasted_sigma[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)
			concatenated_error = np.concatenate([np.concatenate([unbroadcasted_error[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)

			mu.append(np.concatenate(concatenated_mu, axis=0))
			sigma.append(np.concatenate(concatenated_sigma, axis=0))
			error.append(np.concatenate(concatenated_error, axis=0))

			mean_shap.append(np.concatenate(region['shap_dict']['mean_shap'], axis=0))
			std_shap.append(np.concatenate(region['shap_dict']['std_shap'], axis=0))
			X.append(region_scaler.inverse_transform(np.concatenate(region['shap_dict']['xtest'], axis=0)))

mean_shap = np.concatenate(mean_shap, axis=0)
std_shap = np.concatenate(std_shap, axis=0)
X = np.concatenate(X, axis=0)
mu = np.concatenate(mu, axis=0)
sigma = np.concatenate(sigma, axis=0)
error = np.concatenate(error, axis=0)

mean_shap_df = pd.DataFrame(mean_shap, columns=shap_values['features'])
std_shap_df = pd.DataFrame(std_shap, columns=shap_values['features'])
transformed_X_df = pd.DataFrame(X, columns=shap_values['features'])
mu_df = pd.DataFrame(mu, columns=shap_values['features'])
sigma_df = pd.DataFrame(sigma, columns=shap_values['features'])
error_df = pd.DataFrame(error, columns=shap_values['features'])



In [ ]:
feature = ['Vx']

fig, axes = plt.subplots(1, len(feature), figsize=(15, 10))
for i in range(len(feature)):
	axes.hist2d(sigma_df[feature[i]], error_df[feature[i]], bins=100, cmap='viridis', norm=mpl.colors.LogNorm())
	axes.set_title('sigma v error')
	axes.set_xlabel('sigma')
	axes.set_ylabel('error')
plt.savefig('plots/swmag_sigma_vs_error.png')

In [ ]:
feature = ['dbht_median', 'MAGNITUDE_median']

fig, axes = plt.subplots(1, len(feature), figsize=(15, 10))
for i, ax in enumerate(axes):
	axes[i].hist2d(transformed_X_df[feature[i]], mean_shap_df[feature[i]], bins=100, cmap='viridis', norm=mpl.colors.LogNorm())
	ax.set_title(f'{feature[i]}')
	ax.set_xlabel(feature[i])
	ax.set_ylabel('SHAP value')
plt.savefig(f'plots/{feature[0]}_and_{feature[1]}')

In [ ]:
feature = 'cosMLT'

vmin = np.min([np.min(std_shap_df[feature]), np.min(mean_shap_df[feature])])
vmax = np.max([np.max(std_shap_df[feature]),np.max(mean_shap_df[feature])])

fig, axes = plt.subplot_mosaic("AB;DD", figsize=(15,10))
axes['A'].hist2d(transformed_X_df[feature], mean_shap_df[feature], bins=100, cmap='viridis', norm=mpl.colors.LogNorm())
axes['B'].hist2d(transformed_X_df[feature], std_shap_df[feature], bins=100, cmap='viridis', norm=mpl.colors.LogNorm())
plt.title(f'{feature}')
axes['A'].set_ylabel('Percent Contribution')
axes['A'].set_xlabel(feature)
axes['A'].set_xlim(np.min(transformed_X_df[feature]), np.max(transformed_X_df[feature]))
axes['A'].set_ylim(vmin, vmax)
axes['A'].set_title('Mean SHAP')

axes['B'].set_xlim(np.min(transformed_X_df[feature]), np.max(transformed_X_df[feature]))
axes['B'].set_ylim(vmin, vmax)
axes['B'].set_xlabel(feature)
axes['B'].set_ylabel('Percent Contribution')
axes['B'].set_title('STD SHAP')

axes['D'].set_title('Comparison')
axes['D'].hist(mean_shap_df[feature], bins=100, log=True, histtype='step', label='mean')
axes['D'].hist(std_shap_df[feature], bins=100, log=True, histtype='step', label='std')
axes['D'].set_ylabel('log count')
axes['D'].set_xlabel('Percent Contribution')
axes['D'].axvline(0, linestyle='--', color='k')
axes['D'].legend()

plt.tight_layout()
plt.savefig(f'plots/{feature}_swmag_shap_value_distributions.png')

# getting area under the curve of the 1D histograms
from sklearn.metrics import auc
mean_hist, mean_bins = np.histogram(mean_shap_df[feature], bins=100)
std_hist, std_bins = np.histogram(std_shap_df[feature], bins=100)
mean_area = auc(mean_bins[:-1], mean_hist)
std_area = auc(std_bins[:-1], std_hist)


In [ ]:
colors = sns.color_palette('tab20')

fig, axes = plt.subplots(2,1, figsize=(15,10))

for i, col in enumerate(shap_values['features']):
	axes[0].hist(mean_shap_df[col], bins=100, log=True, histtype='step', label=col, color=colors[i])
	axes[1].hist(std_shap_df[col], bins=100, log=True, histtype='step', label=col, color=colors[i])

axes[0].set_title('Mean SHAP')
axes[1].set_title('STD SHAP')

axes[0].legend()
plt.show()

In [ ]:
mean_importance_df = mean_shap_df.abs()
std_importance_df = std_shap_df.abs()

mean_mean_df = mean_importance_df.mean(axis=0)
std_mean_df = std_importance_df.mean(axis=0)

mean_median_df = mean_importance_df.median(axis=0)
std_median_df = std_importance_df.median(axis=0)


fig, axes = plt.subplots(1,2,figsize=(20,10))
x = [(i+1)*4 for i in range(len(mean_importance_df.columns))]
x_mean = [j-0.5 for j in x]
x_std = [k+0.5 for k in x]

axes[0].set_title('Importance Mean')
axes[0].bar(x_mean, mean_mean_df.values, label='mean')
axes[0].bar(x_std, std_mean_df.values, label='std')
axes[0].set_xticks(x)
axes[0].set_xticklabels(mean_importance_df.columns.tolist(), rotation=75)
axes[0].legend()

axes[1].set_title('Importance Median')
axes[1].bar(x_mean, mean_median_df.values, label='mean')
axes[1].bar(x_std, std_median_df.values, label='std')
axes[1].set_xticks(x)
axes[1].set_xticklabels(mean_importance_df.columns.tolist(), rotation=75)

plt.tight_layout()
plt.savefig('plots/swmag_feature_importance.png')

In [ ]:
# individual_cluster_to_examine = 'canadian_cluster'
mean_shap, std_shap, X, mu, sigma, error = [], [], [], [], [], []
dataframes_to_make = ['mean_shap_df', 'std_shap_df', 'transformed_X_df', 'mu_df', 'sigma_df', 'error_df']
time_dict = {f'{feature}':{} for feature in shap_values['features']}

for feat in time_dict.keys():
	for df in dataframes_to_make:
		time_dict[feat][df] = pd.DataFrame()

for cluster in cluster_dict.keys():
	for reg, region in cluster_dict[cluster]['regions'].items():
		if 'shap_dict' in region.keys():

			region['shap_dict']['mean_shap'] = np.concatenate([region['shap_dict']['shap_values'][i][0][:,:,:,:] for i in range(len(region['shap_dict']['shap_values']))], axis=0)
			region['shap_dict']['std_shap'] = np.concatenate([region['shap_dict']['shap_values'][i][1][:,:,:,:] for i in range(len(region['shap_dict']['shap_values']))], axis=0)

			for key in region['shap_dict'].keys():
				if isinstance(region['shap_dict'][key], torch.Tensor):
					region['shap_dict'][key] = region['shap_dict'][key].cpu().detach().numpy()

			try:
				region['shap_dict']['mean_shap'] = region['shap_dict']['mean_shap'].reshape(region['shap_dict']['mean_shap'].shape[0], region['shap_dict']['mean_shap'].shape[2], region['shap_dict']['mean_shap'].shape[3])
				region['shap_dict']['std_shap'] = region['shap_dict']['std_shap'].reshape(region['shap_dict']['std_shap'].shape[0], region['shap_dict']['std_shap'].shape[2], region['shap_dict']['std_shap'].shape[3])
				region['shap_dict']['xtest'] = region['shap_dict']['xtest'].reshape(region['shap_dict']['xtest'].shape[0], region['shap_dict']['xtest'].shape[2], region['shap_dict']['xtest'].shape[3])

				mean_added = np.sum(np.sum(np.abs(region['shap_dict']['mean_shap']), axis=1),axis=1)
				region['shap_dict']['mean_shap'] = region['shap_dict']['mean_shap']/mean_added[:,None,None]

				std_added = np.sum(np.sum(np.abs(region['shap_dict']['std_shap']), axis=1),axis=1)
				region['shap_dict']['std_shap'] = region['shap_dict']['std_shap']/std_added[:,None,None]


			except:
				print('We already did this, skipping....')

			region_scaler = scalers[reg]['swmag_scaler']

			unbroadcasted_mu = results[reg]['swmag_results']['predicted_mean'].to_numpy()
			unbroadcasted_sigma = results[reg]['swmag_results']['predicted_std'].to_numpy()
			unbroadcasted_error = np.array(np.abs(results[reg]['swmag_results']['predicted_mean'] - results[reg]['swmag_results']['actual']))

			concatenated_mu = np.concatenate([np.concatenate([unbroadcasted_mu[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)
			concatenated_sigma = np.concatenate([np.concatenate([unbroadcasted_sigma[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)
			concatenated_error = np.concatenate([np.concatenate([unbroadcasted_error[:,np.newaxis]]*30, axis=1)[:,:,np.newaxis]]*14, axis=2)

			for v, var in enumerate(time_dict.keys()):
				time_dict[var]['mean_shap_df'] = pd.concat([time_dict[var]['mean_shap_df'], pd.DataFrame(region['shap_dict']['mean_shap'][:,:,v])], axis=0)
				time_dict[var]['std_shap_df'] = pd.concat([time_dict[var]['std_shap_df'], pd.DataFrame(region['shap_dict']['std_shap'][:,:,v])], axis=0)
				time_dict[var]['transformed_X_df'] = pd.concat([time_dict[var]['transformed_X_df'], \
																pd.DataFrame(np.array([region_scaler.inverse_transform(region['shap_dict']['xtest'][i,:,:]) \
																for i in range(region['shap_dict']['xtest'].shape[0])])[:,:,v])], axis=0)
				time_dict[var]['mu_df'] = pd.concat([time_dict[var]['mu_df'], pd.DataFrame(concatenated_mu[:,:,v])], axis=0)
				time_dict[var]['sigma_df'] = pd.concat([time_dict[var]['sigma_df'], pd.DataFrame(concatenated_sigma[:,:,v])], axis=0)
				time_dict[var]['error_df'] = pd.concat([time_dict[var]['error_df'], pd.DataFrame(concatenated_error[:,:,v])], axis=0)

			print(len(time_dict['Vx']['mean_shap_df'].columns))

			# mu.append(np.concatenate(concatenated_mu, axis=0))
			# sigma.append(np.concatenate(concatenated_sigma, axis=0))
			# error.append(np.concatenate(concatenated_error, axis=0))

			# mean_shap.append(np.concatenate(region['shap_dict']['mean_shap'], axis=0))
			# std_shap.append(np.concatenate(region['shap_dict']['std_shap'], axis=0))
			# X.append(region_scaler.inverse_transform(np.concatenate(region['shap_dict']['xtest'], axis=0)))

time_columns = [f't-{30-i}' for i in range(0,30)]
for var in time_dict.keys():
	for df in time_dict[var]:
		print(time_dict[var][df].shape)
		print(len(time_dict[var][df].columns))
		time_dict[var][df].columns = time_columns
# mean_shap = np.concatenate(mean_shap, axis=0)
# std_shap = np.concatenate(std_shap, axis=0)
# X = np.concatenate(X, axis=0)
# mu = np.concatenate(mu, axis=0)
# sigma = np.concatenate(sigma, axis=0)
# error = np.concatenate(error, axis=0)

# mean_shap_df = pd.DataFrame(mean_shap, columns=shap_values['features'])
# std_shap_df = pd.DataFrame(std_shap, columns=shap_values['features'])
# transformed_X_df = pd.DataFrame(X, columns=shap_values['features'])
# mu_df = pd.DataFrame(mu, columns=shap_values['features'])
# sigma_df = pd.DataFrame(sigma, columns=shap_values['features'])
# error_df = pd.DataFrame(error, columns=shap_values['features'])



In [ ]:
# var = 'BY_GSM'
df = 'std_shap_df'

fig, axes = plt.subplots(5, 3, figsize=(20,20))
for i, var in enumerate(time_dict.keys()):
	k = i %3
	j = i %5
	axes[j,k].bar([i-30 for i in range(0,30)], time_dict[var][df].abs().mean(axis=0))
	axes[j,k].set_title(var)
plt.savefig(f'plots/time_history_{df}.png')

In [ ]:
for cluster in cluster_dict.values():
	for region in cluster['regions'].values():
		region['mean_lat'] = utils.getting_mean_lat(region['stations'])